In [31]:
import pandas as pd
import awswrangler as awr
import openpyxl


In [32]:


path_to_query = r"C:\Users\raphael.almeida\Documents\Projetos\Seguro de Vida Motorista\listagem_mestra_ajustada.sql"

with open (path_to_query,'r') as reading:
    query_readed = reading.read()

df = awr.athena.read_sql_query(query_readed,'silver')

In [33]:
df.info()
len(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131780 entries, 0 to 131779
Data columns (total 28 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   cooperativa      131780 non-null  string
 1   unidade          131776 non-null  string
 2   cliente          131780 non-null  string
 3   cnpj_cpf         131780 non-null  string
 4   matricula        131780 non-null  Int64 
 5   conjunto         131780 non-null  Int64 
 6   coverage         131780 non-null  Int64 
 7   beneficio        131780 non-null  string
 8   id_driver        131780 non-null  Int64 
 9   id_people        131780 non-null  Int64 
 10  motorista        13706 non-null   string
 11  id_placa         88728 non-null   Int64 
 12  placa            108551 non-null  string
 13  chassi           88728 non-null   string
 14  renavam          88715 non-null   string
 15  ativacao         131774 non-null  object
 16  inicio_vig       131774 non-null  object
 17  fim_vig   

131780

placas totais

In [34]:
df['placa'].drop_duplicates().count()

31761

seguro de vida geral

In [35]:
df_seguro_de_vida = df[df['benefits_id'].isin([6,28,44])]
len(df_seguro_de_vida)

#print(len(df_seguro_de_vida))
#df_seguro_de_vida.head()
#print(len(df_seguro_de_vida.drop_duplicates(subset='placa')))
#df_seguro_de_vida['placa'].drop_duplicates().count()

12262

seguro de vida de motorista

In [36]:
df_seguro_de_vida_motorista = df_seguro_de_vida[df_seguro_de_vida['faixa_valor'].str.contains('MOTORISTA', na=False)]
len(df_seguro_de_vida_motorista)

9712

In [37]:
df_cascos_veiculo = df[df['benefits_id'].isin([2, 24, 40])]
len(df_cascos_veiculo)

17267

In [38]:
df_cascos_veiculo_empresa = df_cascos_veiculo[df_cascos_veiculo['cooperativa']=='Viavante']
df_cascos_veiculo_empresa.head()

,cooperativa,unidade,cliente,cnpj_cpf,matricula,conjunto,coverage,beneficio,id_driver,id_people,...,status,status_coverage,benefits_id,categoria,tipo_categoria,faixa_valor,tabela_ativa,tipo_veiculo,tipo,e_mail
40552,Viavante,UNIDADE VILHENA,J DE QUADROS D AVILA,24320817,4874,7127,52782,CASCO (VEÍCULO),0,0,...,ATIVO,ATIVO,40,CAVALO MECÂNICO,CP: 4% - IVECO - TODOS OS ANOS,"De R$200.000,01 até R$225.000,00",N,Pesado - Cavalo,Novo,<NA>
40554,Viavante,UNIDADE VILHENA,J DE QUADROS D AVILA,24320817,4874,7127,52777,CASCO (VEÍCULO),0,0,...,ATIVO,ATIVO,40,CAVALO MECÂNICO,CP: 4% - SCANIA - TODOS OS ANOS,"De R$975.000,01 até R$1.000.000,00",N,Pesado - Cavalo,Novo,<NA>
40557,Viavante,UNIDADE VILHENA,J DE QUADROS D AVILA,24320817,4874,7127,52772,CASCO (VEÍCULO),0,0,...,ATIVO,ATIVO,40,CAVALO MECÂNICO,CP: 4% - SCANIA - TODOS OS ANOS,"De R$975.000,01 até R$1.000.000,00",N,Pesado - Cavalo,Novo,<NA>
40561,Viavante,UNIDADE JOINVILLE,ALDO TOMASONI,21894930991,4189,7131,52804,CASCO (VEÍCULO),0,0,...,ATIVO,ATIVO,40,3 QUARTOS,CP: 4% - TODAS AS MARCAS - TODOS OS ANOS,"De R$125.000,01 até R$150.000,00",N,Pesado - Três Quartos,Novo,<NA>
40568,Viavante,UNIDADE MARINGÁ,MAE DO CEU TRANSPORTES LTDA,58169086,4878,7138,52851,CASCO (VEÍCULO),0,0,...,ATIVO,ATIVO,40,CAVALO MECÂNICO,CP: 4% - VOLVO - TODOS OS ANOS,"De R$900.000,01 até R$925.000,00",N,Pesado - Cavalo,Novo,<NA>


In [39]:
df_cascos_sem_motorista = df_cascos_veiculo[df_cascos_veiculo['motorista'].notna()]
len(df_cascos_sem_motorista)

0

In [40]:
df_com_motorista = df[df['motorista'].notna()]
df_com_motorista['beneficio'].unique()
#df_empresa = df_com_motorista[df_com_motorista['cooperativa']=='Viavante']
#df_empresa.head()

<StringArray>
[                 'SEGURO DE VIDA',                      'RASTREADOR',
 'RASTREADOR REBOQUE/SEMIRREBOQUE',                    'RASTREAMENTO',
                        'ANUIDADE', 'T4S - RASTREADOR / IMOBILIZADOR',
             'INSERIDO NA APOLICE',                          'VIDROS']
Length: 8, dtype: string

In [41]:
df['id_driver'].value_counts()

id_driver
0    131780
Name: count, dtype: Int64

In [46]:
df_filter1= df_com_motorista[
    (df_com_motorista['cooperativa'] == 'Viavante') &
    (df_com_motorista['beneficio'] != 'SEGURO DE VIDA')
]

df_filter1[['cliente','conjunto','placa','motorista','id_people']].head()

df_filter12 = df_filter1[df_filter1['cliente'].str.contains('SERGIO HENRIQUE COSTA')]

df_filter12[['cliente','conjunto','placa','motorista','id_people']].head()

,cliente,conjunto,placa,motorista,id_people
110625,SERGIO HENRIQUE COSTA E CIA LTDA,14064,TDQ6J66,RONISSON LOPES SIPRIANO,6579
118765,SERGIO HENRIQUE COSTA E CIA LTDA,14064,QCO0H17,RONISSON LOPES SIPRIANO,6579


base_analise

In [4]:

#excluir coluna de e-mail pois está diferenciando na análise
#df = df.drop('e_mail', axis=1)
df = df.drop_duplicates()

#pegar apenas os conjuntos obrigatorios de seguro de vida
beneficios_conjuntos = df.groupby(['conjunto', 'cooperativa'])['benefits_id'].unique()
conjuntos_validos = beneficios_conjuntos[(beneficios_conjuntos.apply(lambda x: {2,6}.issubset((x))))|
                                         (beneficios_conjuntos.apply(lambda x: {24,28}.issubset((x))))|
                                         (beneficios_conjuntos.apply(lambda x: {40,44}.issubset((x))))].index

df_obrigatorio = df[df.set_index(['conjunto','cooperativa']).index.isin(conjuntos_validos)].copy()

#criar colunas para posterior agregação
df_obrigatorio['casco'] = df_obrigatorio['benefits_id'].apply(lambda x: x if (x == 2 or x == 24 or x == 40) else pd.NA)
#df_obrigatorio['seg_vida'] = df_obrigatorio['benefits_id'].apply(lambda x: x if (x==6 or x==28 or x==44) else pd.NA)


df_obrigatorio['seg_vida_motorista'] = df_obrigatorio.apply(lambda row: 
                                                  row['benefits_id'] if (row['benefits_id'] in [6,28,44] and 'MOTORISTA' in row['faixa_valor'])
else pd.NA, axis=1)

df_obrigatorio['seg_vida_geral'] = df_obrigatorio.apply(lambda row: 
                                                  row['benefits_id'] if (row['benefits_id'] in [6,28,44] and 'MOTORISTA' not in row['faixa_valor'])
else pd.NA, axis=1)

#filtrando apenas os que fazem sentido para a análise (casco e seguro de vida)
df_obrigatorio_filtrado = df_obrigatorio[df_obrigatorio['benefits_id'].isin([2,6,24,28,40,44])]

In [5]:
df_obrigatorio_filtrado.head()

,cooperativa,unidade,cliente,matricula,conjunto,coverage,beneficio,motorista,id_placa,placa,...,categoria,tipo_categoria,faixa_valor,tabela_ativa,tipo_veiculo,tipo,e_mail,casco,seg_vida_motorista,seg_vida_geral
0,Stcoop,UNIDADE SINOP,FABIO TURRA JAEGER,21573,13608,84648,CASCO (VEÍCULO),<NA>,17310,RLJ1C15,...,CAVALO,LC: 100% - CP: 4% - SCANIA - 2024 ATÉ 2021,"De R$600.000,01 até R$625.000,00",S,Cavalo,RenovaÃ§Ã£o,<NA>,24,<NA>,<NA>
1,Stcoop,UNIDADE CASCAVEL,RICARDO ROSA AMORIM EIRELI,21578,13610,84655,CASCO (VEÍCULO),<NA>,19062,QAY6H04,...,CAVALO,LC: 100% - CP: 4% - SCANIA - 2024 ATÉ 2021,"De R$625.000,01 até R$650.000,00",S,Cavalo,RenovaÃ§Ã£o,<NA>,24,<NA>,<NA>
5,Stcoop,UNIDADE RONDONÓPOLIS,VILMAR ALEXANDRE DA SILVA TRANSPORTES,11931,13612,84672,SEGURO DE VIDA,VILMAR ALEXANDRE DA SILVA,<NA>,<NA>,...,PADRÃO,PADRÃO,MOTORISTA - Seguro de Vida via Rep. ou Rep. do...,N,<NA>,RenovaÃ§Ã£o,<NA>,<NA>,28,<NA>
8,Stcoop,UNIDADE CURITIBA,A. SAID TRANSPORTES LTDA - ME,6400,13616,84691,CASCO (VEÍCULO),<NA>,18914,BDV1A44,...,CAVALO,LC: 100% - CP: 4% - DAF E MAN - 2024 ATÉ 2019,"De R$350.000,01 até R$375.000,00",S,Cavalo,RenovaÃ§Ã£o,<NA>,24,<NA>,<NA>
10,Stcoop,UNIDADE ITAJAI,SANDRO LUIZ VIEIRA,2312,13618,84709,SEGURO DE VIDA,SANDRO LUIZ VIEIRA,<NA>,<NA>,...,PADRÃO,PADRÃO,MOTORISTA - Seguro de Vida via Rep. ou Rep. do...,N,<NA>,RenovaÃ§Ã£o,<NA>,<NA>,28,<NA>


In [7]:
df_obrigatorio_filtrado['seg_vida_motorista'].count()

9611

conjuntos_analise

In [14]:


df_analise = df_obrigatorio_filtrado.groupby(['cooperativa','unidade','matricula','conjunto','e_mail']).agg(
    qtd_seg_vida_motorista = ('seg_vida_motorista', 'count'),
    qtd_casco = ('casco', 'count'),
    qtd_seg_vida_geral = ('seg_vida_geral', 'count')
).reset_index()

#df_analise.drop_duplicates('e_mail', keep='first')
df_analise.sort_values(by='conjunto', ascending=False)

# df_analysis=df_analise[(df_analise['qtd_motorista']!=df_analise['qtd_casco'])|
#                       (df_analise['qtd_motorista']!=df_analise['qtd_seg_vida'])|
#                       (df_analise['qtd_casco']!=df_analise['qtd_seg_vida'])]

df_analysis = df_analise[df_analise['qtd_seg_vida_motorista']<df_analise['qtd_casco']]

#df_analysis = df_analise[row, ['cooperativa','unidade','matricula','conjunto','qtd_motoristas','qtd_cascos','qtd_segvida']]

# file_path  = r'C:\Users\raphael.almeida\Documents\Projetos\Seguro de Vida Motorista\conjuntos_analise.xlsx'
# df_analysis.to_excel(file_path, engine = 'openpyxl', index = False)

# file_path  = r'C:\Users\raphael.almeida\Documents\Projetos\Seguro de Vida Motorista\conjuntos_variados.xlsx'
# df_analise.to_excel(file_path, engine = 'openpyxl', index = False)

